# Graph

---

## Create the Graph Schema
### Setting Up the Environment Variables
You can either use the following code or set them directly using the `export` command in your shell.

In [1]:
import os
os.environ["TG_HOST"] = "http://127.0.0.1"
os.environ["TG_USERNAME"] = "tigergraph"
os.environ["TG_PASSWORD"] = "tigergraph"

### Create Schema

In [2]:
graph_schema = {
    "graph_name": "Social",
    "nodes": {
        "Person": {
            "primary_key": "name",
            "attributes": {
                "name": "STRING",
                "age": "UINT",
                "gender": "STRING",
            },
        },
    },
    "edges": {
        "Friendship": {
            "is_directed_edge": False,
            "from_node_type": "Person",
            "to_node_type": "Person",
            "attributes": {
                "closeness": "DOUBLE",
            },
        },
    },
}

In [3]:
from tigergraphx import Graph
G = Graph(graph_schema)

2025-02-21 18:56:12,561 - datasets - INFO - PyTorch version 2.6.0 available.
2025-02-21 18:56:12,870 - tigergraphx.core.managers.schema_manager - INFO - Graph existence check for Social: does not exist
2025-02-21 18:56:12,871 - tigergraphx.core.managers.schema_manager - INFO - Creating schema for graph: Social...
2025-02-21 18:56:16,799 - tigergraphx.core.managers.schema_manager - INFO - Graph schema created successfully.


### Retrieve a Graph from the Database

In [4]:
from tigergraphx import Graph
G = Graph.from_db("Social")

## NodeView
If your graph contains only one node type, you don’t need to specify the type when accessing nodes:

In [5]:
>>> G = Graph.from_db("Social")
>>> G.add_nodes_from(["Alice", "Mike"])
>>> len(G.nodes)

2

In [6]:
>>> G.nodes["Alice"]
{'name': 'Alice', 'age': 0, 'gender': ''}
>>> "Alice" in G.nodes

True

In [7]:
>>> G.clear()

True

For graphs with multiple node types, you must include the node type when accessing nodes:

In [8]:
>>> G = Graph.from_db("Social")
>>> G.add_nodes_from(["Alice", "Mike"], "Person")
>>> len(G.nodes)

2

In [9]:
>>> G.nodes[("Person", "Alice")]
{'name': 'Alice', 'age': 0, 'gender': ''}
>>> ("Person", "Alice") in G.nodes

True

In [10]:
>>> G.clear()

True

---

## Schema Operations
### get_schema

In [11]:
>>> G = Graph.from_db("Social")
>>> G.get_schema()

{'graph_name': 'Social',
 'nodes': {'Person': {'primary_key': 'name',
   'attributes': {'name': {'data_type': <DataType.STRING: 'STRING'>,
     'default_value': None},
    'age': {'data_type': <DataType.UINT: 'UINT'>, 'default_value': None},
    'gender': {'data_type': <DataType.STRING: 'STRING'>,
     'default_value': None}},
   'vector_attributes': {}}},
 'edges': {'Friendship': {'is_directed_edge': False,
   'from_node_type': 'Person',
   'to_node_type': 'Person',
   'discriminator': set(),
   'attributes': {'closeness': {'data_type': <DataType.DOUBLE: 'DOUBLE'>,
     'default_value': None}}}}}

To retrieve the schema in JSON format, you can use:

In [12]:
>>> G = Graph.from_db("Social")
>>> G.get_schema("json")

'{"graph_name":"Social","nodes":{"Person":{"primary_key":"name","attributes":{"name":{"data_type":"STRING","default_value":null},"age":{"data_type":"UINT","default_value":null},"gender":{"data_type":"STRING","default_value":null}},"vector_attributes":{}}},"edges":{"Friendship":{"is_directed_edge":false,"from_node_type":"Person","to_node_type":"Person","discriminator":[],"attributes":{"closeness":{"data_type":"DOUBLE","default_value":null}}}}}'

### create_schema
If you do not need to drop the existing graph before creating the schema, you can use:

In [13]:
>>> G = Graph.from_db("Social")
>>> G.create_schema()

2025-02-21 18:56:33,988 - tigergraphx.core.managers.schema_manager - INFO - Graph existence check for Social: exists
2025-02-21 18:56:33,989 - tigergraphx.core.managers.schema_manager - INFO - Graph 'Social' already exists. Skipping graph creation.


False

If you need to drop the existing graph, you can call:

In [14]:
>>> G = Graph.from_db("Social")
>>> G.create_schema(True)

2025-02-21 18:56:35,677 - tigergraphx.core.managers.schema_manager - INFO - Graph existence check for Social: exists
2025-02-21 18:56:35,678 - tigergraphx.core.managers.schema_manager - INFO - Dropping graph: Social...
2025-02-21 18:56:38,492 - tigergraphx.core.managers.schema_manager - INFO - Graph dropped successfully.
2025-02-21 18:56:38,493 - tigergraphx.core.managers.schema_manager - INFO - Creating schema for graph: Social...
2025-02-21 18:56:41,448 - tigergraphx.core.managers.schema_manager - INFO - Graph schema created successfully.


True

### drop_graph

In [15]:
>>> G = Graph.from_db("Social")
>>> G.drop_graph()

2025-02-21 18:56:43,112 - tigergraphx.core.managers.schema_manager - INFO - Dropping graph: Social...
2025-02-21 18:56:46,457 - tigergraphx.core.managers.schema_manager - INFO - Graph dropped successfully.


---

## Data Loading Operations
### load_data

In [16]:
>>> G.create_schema(True)

2025-02-21 18:56:48,718 - tigergraphx.core.managers.schema_manager - INFO - Graph existence check for Social: does not exist
2025-02-21 18:56:48,719 - tigergraphx.core.managers.schema_manager - INFO - Creating schema for graph: Social...
2025-02-21 18:56:52,931 - tigergraphx.core.managers.schema_manager - INFO - Graph schema created successfully.


True

In [17]:
loading_job_config = {
    "loading_job_name": "loading_job_Social",
    "files": [
        {
            "file_alias": "f_person",
            "file_path": "/home/tigergraph/data/person_data.csv",
            "csv_parsing_options": {
                "separator": ",",
                "header": True,
                "EOL": "\\n",
                "quote": "DOUBLE",
            },
            "node_mappings": [
                {
                    "target_name": "Person",
                    "attribute_column_mappings": {
                        "name": "name",
                        "age": "age",
                    },
                }
            ],
        },
        {
            "file_alias": "f_friendship",
            "file_path": "/home/tigergraph/data/friendship_data.csv",
            "edge_mappings": [
                {
                    "target_name": "Friendship",
                    "source_node_column": "source",
                    "target_node_column": "target",
                    "attribute_column_mappings": {
                        "closeness": "closeness",
                    },
                }
            ],
        },
    ],
}

In [18]:
>>> G = Graph.from_db("Social")
>>> G.load_data(loading_job_config)

2025-02-21 18:56:55,701 - tigergraphx.core.managers.data_manager - INFO - Initiating data load for job: loading_job_Social...
2025-02-21 18:57:03,561 - tigergraphx.core.managers.data_manager - INFO - Data load completed successfully.


In [19]:
>>> print(G.number_of_nodes())

1


In [20]:
>>> print(G.number_of_edges())

1


In [21]:
>>> G.clear()

True

---

## Node Operations
### add_node
Single Node Type Example:

In [22]:
>>> G = Graph.from_db("Social")
>>> G.add_node("Alice", age=30, gender="Female")
>>> G.add_node("Mike", age=29)
>>> len(G.nodes)

2

In [23]:
>>> G.clear()

True

Multiple Node Types Example:

In [24]:
>>> G = Graph.from_db("Social")
>>> G.add_node("Alice", "Person", age=30, gender="Female")
>>> G.add_node("Mike", "Person", age=29)
>>> len(G.nodes)

2

In [25]:
>>> G.clear()

True

### add_nodes_from
Single Node Type Example:

In [26]:
>>> G = Graph.from_db("Social")
>>> # Add nodes using a list of node IDs only, without additional attributes
>>> G.add_nodes_from(["Alice", "Mike"])

2

In [27]:
>>> # Add nodes with individual attributes using a list of (ID, attribute_dict) tuples
>>> nodes_for_adding = [
...    ("Alice", {"age": 30, "gender": "Female"}),
...    ("Mike", {"age": 29}),
... ]
>>> G.add_nodes_from(nodes_for_adding)

2

In [28]:
>>> # Add nodes with shared attributes applied to all listed node IDs
>>> G.add_nodes_from(["Alice", "Mike"], age=30)

2

In [29]:
>>> G.clear()

True

Multiple Node Types Example:

In [30]:
>>> G = Graph.from_db("Social")
>>> nodes_for_adding = [
...    ("Alice", {"age": 30, "gender": "Female"}),
...    ("Mike", {"age": 29}),
... ]
>>> G.add_nodes_from(nodes_for_adding, "Person")

2

In [31]:
>>> G.clear()

True

### remove_node
Single Node Type Example:

In [32]:
>>> G = Graph.from_db("Social")
>>> G.add_node("Alice", age=30, gender="Female")
>>> len(G.nodes)

1

In [33]:
>>> G.remove_node("Alice")

True

In [34]:
>>> len(G.nodes)

0

Multiple Node Types Example:

In [35]:
>>> G = Graph.from_db("Social")
>>> G.add_node("Alice", "Person", age=30, gender="Female")
>>> len(G.nodes)

1

In [36]:
>>> G.remove_node("Alice", "Person")

True

In [37]:
>>> len(G.nodes)

0

### has_node
Single Node Type Example:

In [38]:
>>> G = Graph.from_db("Social")
>>> G.add_node("Alice", age=30, gender="Female")
>>> G.has_node("Alice")

True

In [39]:
>>> G.clear()

True

Multiple Node Types Example:

In [40]:
>>> G = Graph.from_db("Social")
>>> G.add_node("Alice", "Person", age=30, gender="Female")
>>> G.has_node("Alice", "Person")

True

In [41]:
>>> G.clear()

True

### get_node_data

In [42]:
>>> G = Graph.from_db("Social")
>>> G.add_node("Alice", age=30, gender="Female")
>>> G.get_node_data("Alice")

{'name': 'Alice', 'age': 30, 'gender': 'Female'}

In [43]:
>>> G.clear()

True

Multiple Node Types Example:

In [44]:
>>> G = Graph.from_db("Social")
>>> G.add_node("Alice", "Person", age=30, gender="Female")
>>> G.get_node_data("Alice", "Person")

{'name': 'Alice', 'age': 30, 'gender': 'Female'}

In [45]:
>>> G.clear()

True

### get_node_edges
Single Node Type Example:

In [46]:
>>> G = Graph.from_db("Social")
>>> nodes_for_adding = [
...    ("Alice", {"age": 30, "gender": "Female"}),
...    ("Mike", {"age": 29}),
... ]
>>> G.add_nodes_from(nodes_for_adding)

2

In [47]:
>>> G.add_edges_from([("Alice", "Mike")])

1

In [48]:
>>> G.get_node_edges("Alice")

[('Alice', 'Mike')]

In [49]:
>>> G.clear()

True

Multiple Node Types Example:

In [50]:
>>> G = Graph.from_db("Social")
>>> nodes_for_adding = [
...    ("Alice", {"age": 30, "gender": "Female"}),
...    ("Mike", {"age": 29}),
... ]
>>> G.add_nodes_from(nodes_for_adding, "Person")

2

In [51]:
>>> G.add_edges_from([("Alice", "Mike")], "Person", "Friendship", "Person")

1

In [52]:
>>> # Retrieve all edges connected to Alice, regardless of type
>>> G.get_node_edges("Alice", "Person")

[('Alice', 'Mike')]

In [53]:
>>> # Retrieve only edges of type "Friendship"
>>> G.get_node_edges("Alice", "Person", "Friendship")

[('Alice', 'Mike')]

In [54]:
>>> # Retrieve edges of multiple specified types.
>>> G.get_node_edges("Alice", "Person", ["Friendship", "Friendship"]) 

[('Alice', 'Mike')]

In [55]:
>>> G.clear()

True

### clear

In [56]:
>>> G = Graph.from_db("Social")
>>> nodes_for_adding = [
...    ("Alice", {"age": 30, "gender": "Female"}),
...    ("Mike", {"age": 29}),
... ]
>>> G.add_nodes_from(nodes_for_adding, "Person")

2

In [57]:
>>> len(G.nodes)

2

In [58]:
>>> G.clear()

True

In [59]:
>>> len(G.nodes)

0

---

## Edge Operations
### add_edge
Single Node Type Example:

In [60]:
>>> G = Graph.from_db("Social")
>>> nodes_for_adding = [
...    ("Alice", {"age": 30, "gender": "Female"}),
...    ("Mike", {"age": 29}),
... ]
>>> G.add_nodes_from(nodes_for_adding)

2

In [61]:
>>> G.add_edge("Alice", "Mike", closeness=2.5)
>>> G.has_edge("Alice", "Mike")

True

In [62]:
>>> G.clear()

True

Multiple Node Types Example:

In [63]:
>>> G = Graph.from_db("Social")
>>> nodes_for_adding = [
...    ("Alice", {"age": 30, "gender": "Female"}),
...    ("Mike", {"age": 29}),
... ]
>>> G.add_nodes_from(nodes_for_adding, "Person")

2

In [64]:
>>> G.add_edge("Alice", "Mike", "Person", "Friendship", "Person", closeness=2.5)
>>> G.has_edge("Alice", "Mike", "Person", "Friendship", "Person")

True

In [65]:
>>> G.clear()

True

### add_edges_from
Single Node Type Example:

In [66]:
>>> G = Graph.from_db("Social")
>>> nodes_for_adding = [
...    ("Alice", {"age": 30, "gender": "Female"}),
...    ("Mike", {"age": 29}),
... ]
>>> G.add_nodes_from(nodes_for_adding)

2

In [67]:
>>> # Add edges using a list of (source ID, target ID) tuples, without attributes
>>> G.add_edges_from([("Alice", "Mike"), ("Alice", "John")])

2

In [68]:
>>> # Add edges with individual attributes using (source ID, target ID, attribute_dict) tuples
>>> ebunch_to_add = [
...    ("Alice", "Mike"),
...    ("Alice", "John", {"closeness": 2.5}),
... ]
>>> G.add_edges_from(ebunch_to_add)

2

In [69]:
>>> # Add edges with shared attributes applied to all listed edges
>>> G.add_edges_from([("Alice", "Mike"), ("Alice", "John")], closeness=2.5)

2

In [70]:
>>> G.clear()

True

Multiple Node Types Example:

In [71]:
>>> G = Graph.from_db("Social")
>>> nodes_for_adding = [
...    ("Alice", {"age": 30, "gender": "Female"}),
...    ("Mike", {"age": 29}),
... ]
>>> G.add_nodes_from(nodes_for_adding, "Person")

2

In [72]:
>>> G.add_edges_from([("Alice", "Mike")], "Person", "Friendship", "Person")

1

In [73]:
>>> G.clear()

True

### has_edge
Single Node Type Example:

In [74]:
>>> G = Graph.from_db("Social")
>>> nodes_for_adding = [
...    ("Alice", {"age": 30, "gender": "Female"}),
...    ("Mike", {"age": 29}),
... ]
>>> G.add_nodes_from(nodes_for_adding)

2

In [75]:
>>> G.add_edge("Alice", "Mike")
>>> G.has_edge("Alice", "Mike")

True

In [76]:
>>> G.clear()

True

Multiple Node Types Example:

In [77]:
>>> G = Graph.from_db("Social")
>>> nodes_for_adding = [
...    ("Alice", {"age": 30, "gender": "Female"}),
...    ("Mike", {"age": 29}),
... ]
>>> G.add_nodes_from(nodes_for_adding, "Person")

2

In [78]:
>>> G.add_edge("Alice", "Mike", "Person", "Friendship", "Person")
>>> G.has_edge("Alice", "Mike", "Person", "Friendship", "Person")

True

In [79]:
>>> G.clear()

True

### get_edge_data
Single Node Type Example:

In [80]:
>>> G = Graph.from_db("Social")
>>> nodes_for_adding = [
...    ("Alice", {"age": 30, "gender": "Female"}),
...    ("Mike", {"age": 29}),
... ]
>>> G.add_nodes_from(nodes_for_adding)

2

In [81]:
>>> G.add_edge("Alice", "Mike", closeness=2.5)
>>> G.get_edge_data("Alice", "Mike")

{'closeness': 2.5}

In [82]:
>>> G.clear()

True

Multiple Node Types Example:

In [83]:
>>> G = Graph.from_db("Social")
>>> nodes_for_adding = [
...    ("Alice", {"age": 30, "gender": "Female"}),
...    ("Mike", {"age": 29}),
... ]
>>> G.add_nodes_from(nodes_for_adding, "Person")

2

In [84]:
>>> G.add_edge("Alice", "Mike", "Person", "Friendship", "Person", closeness=2.5)
>>> G.get_edge_data("Alice", "Mike", "Person", "Friendship", "Person")

{'closeness': 2.5}

In [85]:
>>> G.clear()

True

---

## Statistics Operations
### degree
Single Node Type Example:

In [86]:
>>> G = Graph.from_db("Social")
>>> nodes_for_adding = [
...    ("Alice", {"age": 30, "gender": "Female"}),
...    ("Mike", {"age": 29}),
... ]
>>> G.add_nodes_from(nodes_for_adding)

2

In [87]:
>>> G.add_edges_from([("Alice", "Mike")])

1

In [88]:
>>> G.degree("Alice")

1

In [89]:
>>> G.clear()

True

Multiple Node Types Example:

In [94]:
>>> G = Graph.from_db("Social")
>>> G.add_edges_from([("Alice", "Mike")], "Person", "Friendship", "Person")

1

In [95]:
>>> # Get the degree of node Alice for all edge types
>>> G.degree("Alice", "Person")

1

In [96]:
>>> # Get the degree of node Alice for a single edge type
>>> G.degree("Alice", "Person", "Friendship")

1

In [97]:
>>> # Get the degree of node Alice for multiple specified edge types.
>>> G.degree("Alice", "Person", ["Friendship", "Friendship"])

1

In [98]:
>>> G.clear()

True

### number_of_nodes

In [99]:
>>> G = Graph.from_db("Social")
>>> nodes_for_adding = [
...    ("Alice", {"age": 30, "gender": "Female"}),
...    ("Mike", {"age": 29}),
... ]
>>> G.add_nodes_from(nodes_for_adding, "Person")

2

In [100]:
>>> # Get the total number of edges in the graph
>>> G.number_of_nodes()

2

In [101]:
>>> # Get the number of edges of type "Friendship"
>>> G.number_of_nodes("Person")

2

In [102]:
>>> G.clear()

True

### number_of_edges

In [103]:
>>> G = Graph.from_db("Social")
>>> ebunch_to_add = [
...    ("Alice", "Mike"),
...    ("Alice", "John", {"closeness": 2.5}),
... ]
>>> G.add_edges_from(ebunch_to_add)

2

In [104]:
>>> # Get the total number of edges in the graph
>>> G.number_of_edges()

2

In [105]:
>>> # Get the number of edges of type "Friendship"
>>> G.number_of_edges("Friendship")

2

In [106]:
>>> G.clear()

True

---

## Query Operations

In [107]:
>>> G = Graph.from_db("Social")
>>> nodes_for_adding = [
...    ("Alice", {"age": 30, "gender": "Female"}),
...    ("Mike", {"age": 29, "gender": "Male"}),
...    ("Emily", {"age": 28, "gender": "Female"}),
... ]
>>> G.add_nodes_from(nodes_for_adding, "Person")

3

In [108]:
>>> # Get all nodes of type "Person"
>>> df = G.get_nodes("Person")
>>> print(df)

    v_id  v_type  gender   name  age
0   Mike  Person    Male   Mike   29
1  Alice  Person  Female  Alice   30
2  Emily  Person  Female  Emily   28


In [109]:
>>> # Get all nodes of all types
>>> df = G.get_nodes(all_node_types=True)
>>> print(df)

    v_id  v_type  gender   name  age
0  Emily  Person  Female  Emily   28
1   Mike  Person    Male   Mike   29
2  Alice  Person  Female  Alice   30


In [110]:
>>> # Retrieve nodes with a filter expression
>>> df = G.get_nodes(
...     node_type="Person",
...     node_alias="s", # "s" is the default value, so you can remove this line
...     filter_expression="s.age >= 29",
... )
>>> print(df)

    v_id  v_type  gender   name  age
0   Mike  Person    Male   Mike   29
1  Alice  Person  Female  Alice   30


In [111]:
>>> # Retrieve women aged 29 or older
>>> df = G.get_nodes(node_type="Person", filter_expression='s.age >= 29 and s.gender == "Female"')
>>> print(df)

    v_id  v_type  gender   name  age
0  Alice  Person  Female  Alice   30


In [112]:
>>> # Retrieve only specific attributes
>>> df = G.get_nodes(
...     node_type="Person",
...     return_attributes=["name", "gender"],
... )
>>> print(df)

    name  gender
0  Alice  Female
1  Emily  Female
2   Mike    Male


In [113]:
>>> # Limit the number of nodes returned
>>> df = G.get_nodes(
...     node_type="Person",
...     limit=1,
... )
>>> print(df)

    v_id  v_type  gender   name  age
0  Emily  Person  Female  Emily   28


In [114]:
>>> # Retrieve "Person" nodes with a specific filter expression,
>>> # use a custom alias, request only selected attributes, and limit the results.
>>> df = G.get_nodes(
...     node_type="Person",
...     filter_expression="s.age >= 29",
...     return_attributes=["name", "age"],
...     limit=1
... )
>>> print(df)

   name  age
0  Mike   29


In [115]:
>>> G.clear()

True

### get_neighbors

In [116]:
>>> G = Graph.from_db("Social")
>>> nodes_for_adding = [
...    ("Alice", {"age": 30, "gender": "Female"}),
...    ("Mike", {"age": 29, "gender": "Male"}),
...    ("Emily", {"age": 28, "gender": "Female"}),
...    ("John", {"age": 27, "gender": "Male"}),
...    ("Mary", {"age": 28, "gender": "Female"}),
... ]
>>> G.add_nodes_from(nodes_for_adding, "Person")

5

In [117]:
>>> ebunch_to_add = [
...    ("Alice", "Mike", {"closeness": 1.5}),
...    ("Alice", "John", {"closeness": 2.5}),
...    ("John", "Emily", {"closeness": 3.5}),
...    ("Emily", "Mary", {"closeness": 3.5}),
... ]
>>> G.add_edges_from(ebunch_to_add)

4

In [118]:
>>> # Get neighbors of Alice
>>> df = G.get_neighbors(start_nodes="Alice", start_node_type="Person")
>>> print(df)

  gender  name  age
0   Male  John   27
1   Male  Mike   29


In [119]:
>>> # Get neighbors of Alice with a specific edge type
>>> df = G.get_neighbors(
...     start_nodes="Alice",
...     start_node_type="Person",
...     edge_types="Friendship",
... )
>>> print(df)

  gender  name  age
0   Male  John   27
1   Male  Mike   29


In [120]:
>>> # Get neighbors of Alice with a filter expression
>>> df = G.get_neighbors(
...     start_nodes="Alice",
...     start_node_type="Person",
...     start_node_alias="s", # "s" is the default value, so you can remove this line
...     edge_alias="e", # "e" is the default value, so you can remove this line
...     target_node_alias="t", # "t" is the default value, so you can remove this line
...     filter_expression="e.closeness > 1.5",
... )
>>> print(df)

  gender  name  age
0   Male  John   27


In [121]:
>>> # Retrieve only specific attributes for neighbors
>>> df = G.get_neighbors(
...     start_nodes="Alice",
...     start_node_type="Person",
...     return_attributes=["name", "gender"],
... )
>>> print(df)

   name gender
0  Mike   Male
1  John   Male


In [122]:
>>> # Limit the number of neighbors returned
>>> df = G.get_neighbors(
...     start_nodes="Alice",
...     start_node_type="Person",
...     limit=1,
... )
>>> print(df)

  gender  name  age
0   Male  John   27


In [123]:
>>> # Retrieve the first target node of type "Person" that is a friend of Alice (a "Person"),
>>> # filtering edges by "closeness > 1" and returning the target node's "name" and "gender".
>>> df = G.get_neighbors(
...     start_nodes="Alice",
...     start_node_type="Person",
...     edge_types="Friendship",
...     target_node_types="Person",
...     filter_expression="e.closeness > 1",
...     return_attributes=["name", "gender"],
...     limit=1,
... )
>>> print(df)

   name gender
0  John   Male


In [124]:
>>> G.clear()

True

---
